In [1]:
import numpy as np
import pandas as pd
import requests
import pickle
from bs4 import BeautifulSoup
import jieba
import re
import itertools
import time
import os
import json

In [20]:
# scrapy everything down
cywl_list = ["https://guba.eastmoney.com/list,603569_" + str(i) + ".html" for i in range(1,370)]
cywl_iterator = iter(cywl_list)
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0)Gecko/20100101 Firefox/87.0"}
results = []
count = 1
for url in cywl_iterator:
    print(url)
    ind_not_get_data = True
    try:
        res = requests.get(url,headers = headers,timeout=10)
        time.sleep(np.random.randint(0,3))
        ind_not_get_data = False
        #res.encoding("GB2312")
        print(f"{count}page is done")
        results.append(res)
        count += 1
    except:
        print("Fail because of Timeout")
        url_temp = url

https://guba.eastmoney.com/list,603569_1.html
0页已爬取
https://guba.eastmoney.com/list,603569_2.html
1页已爬取
https://guba.eastmoney.com/list,603569_3.html
2页已爬取
https://guba.eastmoney.com/list,603569_4.html
3页已爬取
https://guba.eastmoney.com/list,603569_5.html
4页已爬取
https://guba.eastmoney.com/list,603569_6.html
5页已爬取
https://guba.eastmoney.com/list,603569_7.html
6页已爬取
https://guba.eastmoney.com/list,603569_8.html
7页已爬取
https://guba.eastmoney.com/list,603569_9.html
8页已爬取
https://guba.eastmoney.com/list,603569_10.html
9页已爬取
https://guba.eastmoney.com/list,603569_11.html
10页已爬取
https://guba.eastmoney.com/list,603569_12.html
11页已爬取
https://guba.eastmoney.com/list,603569_13.html
12页已爬取
https://guba.eastmoney.com/list,603569_14.html
13页已爬取
https://guba.eastmoney.com/list,603569_15.html
14页已爬取
https://guba.eastmoney.com/list,603569_16.html
15页已爬取
https://guba.eastmoney.com/list,603569_17.html
16页已爬取
https://guba.eastmoney.com/list,603569_18.html
17页已爬取
https://guba.eastmoney.com/list,603569_19.html


In [22]:
if not os.path.exists("data"):
    os.makedirs("data")

for i in range(len(results)):
    name = str(i) + ".html"
    with open("data/" + name,"w+",encoding="utf-8") as f:
        res = results[i]
        res.encoding=res.apparent_encoding
        f.write(results[i].text)

In [23]:
# how I get the right RE
""" 
p1 = r'<tr.+?ter">.+?(\d{4}-\d{2}-\d{2}).+?</div.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div>.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div>'
p2 = r'<tr data-index.+?<td class="decs_col".+?<a href=.+?>(.+?)<.+?'
pattern2 = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">(.+?)</a>.+?"author">(.+?)</a>.+?"update">(.+?)</div>'
pattern3 = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">.+?.html">(.+?)</a>.+?"author">(.+?)</a>.+?"update">(.+?)</div>'
pattern4 = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">.+?.html">(.+?)</a>.+?"author">.+?">(.+?)</a>.+?"update">(.+?)</div>'
"""
p = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">.+?.html">(.+?)</a>.+?"author">.+?">(.+?)</a>.+?"update">(.+?)</div>'

with open("10.html","r",encoding = "utf-8") as f:
    res = f.read()
match = re.findall(p,res,re.S)
match

[('13405', '81', '方正证券(601901)龙虎榜数据(12-20)', '方正证券资讯', '12-20 04:54'),
 ('1191',
  '12',
  '这次真走了，明天上午就把资金取出来，然后进行销户流程，接着再把资金买入纳斯达克指',
  '叶少老师永退',
  '12-20 04:47'),
 ('520', '24', '#【悬赏已结束】双十一返场哪些上市公司消费产品值得关注？#', '基民rwYdzV', '12-20 03:56'),
 ('26257',
  '53',
  '没有人去证监会网站举报么？为什么不发公告？符合规定么？中小股东的知情权在哪里？',
  '巴渝公子',
  '12-20 04:08'),
 ('98', '1', '昨天首创跌停，今天华创、方正跌停，明天东方可能要糟了', 'nmeidat', '12-20 05:02'),
 ('2142', '24', '今天又破新低?静待反弹！！', '股友5nx5827712', '12-20 04:16'),
 ('393', '6', '方正罪不至此吧……', '股友N95213G633', '12-20 04:51'),
 ('1514', '43', '最少2个跌停', '股友0n5666I171', '12-20 01:19'),
 ('47457371', '664', '券商板块拉升 方正证券涨停', '方正证券资讯', '11-07 09:50'),
 ('21', '0', '方正证券沪股通', '股友3O288x2621', '12-20 05:32'),
 ('42', '0', '市场走势已经说明了，没有什么利好了', '巴渝公子', '12-12 09:40'),
 ('28696', '178', '方正证券:关于持股5%以上股东权益变动达到1%的提示性公告', '方正证券资讯', '11-23 05:44'),
 ('180', '1', '不理解为什么家人们来接盘，完全不理解', '高抛低吸的九儿', '12-20 05:02'),
 ('94', '2', '我的10个亿都没有挡住停', 'baby喜气羊羊', '12-20 01:58'),
 ('1', '0', '不出消息是故意而为之！就是为了今天这样子！哈哈，对吧！

In [25]:
if not os.path.exists("res"):
    os.makedirs("res")

results = []
num = []
count = 0
for i in range(361):
    name = str(i) + ".html"
    with open(name,"r",encoding="utf-8") as f:
        temp = f.read()
    result = re.findall(p,temp,re.S)
    results += result
res_df = pd.DataFrame(results,columns=["comment","reply","content","author","time"])
#res_df.to_excel("result/cywl_gb.xlsx",encoding="utf-8")
year = 2023
for i in range(len(res_df["time"])-1):
    if res_df["time"][i][:2] == "01" and res_df["time"][i+1][:2] == "12":
        year -= 1
    res_df["time"][i] = str(year) + "-" + res_df["time"][i]
res_df.iloc[-1,4] = str(year) + "-" + res_df.iloc[-1,4]
res_df["time"] = pd.to_datetime(res_df["time"])
res_df["time2"] = pd.to_datetime(res_df["time"].apply(lambda x:str(x)[:10]),format = "%Y-%m-%d")
i1 = res_df["time"].groupby(res_df["time2"]).count()
i2 = res_df["reply"].groupby(res_df["time2"]).mean()
i3 = pd.Series(i1 * i2,name="flc")
res_df.to_excel("res/res.xlsx",encoding="utf-8")

d:\Miniconda\envs\bench_env\Lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
